<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Enhanced_and_Fancy_LAB_Color_Analyzer_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Enhanced and Fancy LAB Color Analyzer 🚀

# ------------------------------
# 1. Install and Import Libraries
# ------------------------------

# Install required libraries only if not already installed
import sys
import subprocess

def install_packages(packages):
    """
    Installs the listed packages using pip if they are not already installed.

    Parameters:
        packages (list): List of package names as strings.
    """
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = ['colormath', 'plotly', 'ipywidgets', 'pandas', 'numpy']
install_packages(required_packages)

# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from google.colab import files
import io
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Enable ipywidgets in Colab
from google.colab import output
output.enable_custom_widget_manager()

# ------------------------------
# 2. Define Utility Functions
# ------------------------------

def upload_dataset():
    """
    Prompts the user to upload the ISCC-NBS LAB colors CSV file.

    Returns:
        pd.DataFrame: DataFrame containing the ISCC-NBS LAB colors.
    """
    print("📂 Please upload your 'iscc_nbs_lab_colors.csv' file.")
    uploaded = files.upload()

    if not uploaded:
        raise FileNotFoundError("No file was uploaded. Please upload 'iscc_nbs_lab_colors.csv'.")

    # List all uploaded files
    uploaded_files = list(uploaded.keys())
    print("📥 Uploaded Files:")
    for file in uploaded_files:
        print(f" - {file}")

    # Check for the expected filename
    expected_filename = 'iscc_nbs_lab_colors.csv'
    if expected_filename in uploaded_files:
        df = pd.read_csv(io.BytesIO(uploaded[expected_filename]))
        print(f"✅ '{expected_filename}' uploaded successfully.")
        return df
    else:
        # If expected file not found, prompt user to select from uploaded files
        print(f"❌ Expected file '{expected_filename}' not found.")
        print("🔍 Please select one of the uploaded CSV files:")
        for idx, file in enumerate(uploaded_files, start=1):
            print(f"{idx}. {file}")

        # Prompt user to input the number corresponding to their file
        try:
            selected_idx = int(input("Enter the number of the file to use: ")) - 1
            if selected_idx < 0 or selected_idx >= len(uploaded_files):
                raise ValueError("Invalid selection.")
        except (ValueError, IndexError):
            raise ValueError("Invalid selection. Please restart the process and select a valid file.")

        selected_file = uploaded_files[selected_idx]
        df = pd.read_csv(io.BytesIO(uploaded[selected_file]))
        print(f"✅ '{selected_file}' uploaded successfully.")
        return df

def validate_dataset(df):
    """
    Validates that the DataFrame contains the necessary columns and no missing values.

    Parameters:
        df (pd.DataFrame): The DataFrame to validate.

    Raises:
        ValueError: If required columns are missing or contain non-numeric values.
    """
    # Define required columns as a list for indexing
    required_columns = ['L', 'A', 'B', 'Color Name']

    # Check for missing columns
    missing = set(required_columns) - set(df.columns)
    if missing:
        raise ValueError(f"CSV file is missing required columns: {missing}")

    # Check for missing values in required columns
    missing_values = df[required_columns].isnull().sum()
    if missing_values.any():
        print("❌ Missing values detected in the following columns:")
        print(missing_values[missing_values > 0])
        # Display rows with missing values
        print("\n📄 Displaying rows with missing values:")
        display(df[df[required_columns].isnull().any(axis=1)])
        raise ValueError("CSV file contains missing values in required columns.")

    # Ensure 'L', 'A', 'B' columns are numeric
    for col in ['L', 'A', 'B']:
        if not pd.api.types.is_numeric_dtype(df[col]):
            raise ValueError(f"Column '{col}' must contain numeric values.")

    # Optionally, convert columns to float if they are not
    df['L'] = pd.to_numeric(df['L'], errors='coerce')
    df['A'] = pd.to_numeric(df['A'], errors='coerce')
    df['B'] = pd.to_numeric(df['B'], errors='coerce')

    # After conversion, check again for NaN
    if df[['L', 'A', 'B']].isnull().values.any():
        print("❌ Missing or invalid numeric values detected after conversion:")
        print(df[['L', 'A', 'B']].isnull().sum())
        raise ValueError("Conversion to numeric failed for some rows in 'L', 'A', or 'B' columns.")

def calculate_delta_e(input_lab, dataset_df):
    """
    Vectorized Delta-E (CIE76) calculation between input LAB and dataset LAB colors.

    Parameters:
        input_lab (np.ndarray): The input LAB color as [L, A, B].
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.

    Returns:
        np.ndarray: Array of Delta-E values.
    """
    delta_e = np.linalg.norm(dataset_df[['L', 'A', 'B']].values - input_lab, axis=1)
    return delta_e

def find_closest_color(input_lab, dataset_df):
    """
    Finds the closest color in the dataset to the input LAB color based on Delta-E.

    Parameters:
        input_lab (np.ndarray): The input LAB color as [L, A, B].
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.

    Returns:
        pd.Series: Closest color row.
        float: Minimum Delta-E value.
    """
    delta_e_values = calculate_delta_e(input_lab, dataset_df)

    # Check if delta_e_values contains valid numbers
    if np.all(np.isnan(delta_e_values)):
        raise ValueError("Delta-E calculation resulted in all NaN values. Check your dataset and input LAB values.")

    min_idx = np.nanargmin(delta_e_values)  # Use nanargmin to ignore NaNs
    min_delta_e = delta_e_values[min_idx]
    closest_color = dataset_df.iloc[min_idx]
    return closest_color, min_delta_e

def lab_to_rgb(lab_color):
    """
    Converts a LAB color to RGB.

    Parameters:
        lab_color (list or tuple): The LAB color as [L, A, B].

    Returns:
        tuple: RGB color as (R, G, B) with values between 0 and 255.
    """
    try:
        lab = LabColor(lab_l=lab_color[0], lab_a=lab_color[1], lab_b=lab_color[2])
        rgb = convert_color(lab, sRGBColor, target_illuminant='d65')
        rgb_clamped = (
            int(max(0, min(rgb.rgb_r, 1)) * 255),
            int(max(0, min(rgb.rgb_g, 1)) * 255),
            int(max(0, min(rgb.rgb_b, 1)) * 255)
        )
        return rgb_clamped
    except Exception as e:
        print(f"❌ Error converting LAB to RGB: {e}")
        return (0, 0, 0)

def validate_lab_color(lab):
    """
    Validates the input LAB color.

    Parameters:
        lab (list or tuple): The LAB color as [L, A, B].

    Raises:
        ValueError: If the LAB color is invalid.
    """
    if not isinstance(lab, (list, tuple, np.ndarray)) or len(lab) != 3:
        raise ValueError("Input LAB color must be a list, tuple, or array of three numerical values.")
    L, A, B = lab
    if not (0 <= L <= 100):
        raise ValueError("L component must be between 0 and 100.")
    if not (-128 <= A <= 127):
        raise ValueError("A component must be between -128 and 127.")
    if not (-128 <= B <= 127):
        raise ValueError("B component must be between -128 and 127.")

# ------------------------------
# 3. Define Visualization Functions
# ------------------------------

def create_advanced_color_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_color_name, delta_e):
    """
    Creates an enhanced Plotly Express scatter plot displaying input and closest colors with annotations.

    Parameters:
        input_rgb (tuple): RGB values of the input color.
        closest_rgb (tuple): RGB values of the closest match.
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.
        delta_e (float): Delta-E value between input and closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    # Prepare data
    data = pd.DataFrame({
        'Color Type': ['Input Color', f'Closest: {closest_color_name}'],
        'RGB': [f'rgb{input_rgb}', f'rgb{closest_rgb}'],
        'LAB': [f'L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}',
                f'L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}'],
        'Delta-E': [delta_e, 'N/A']
    })

    # Create scatter plot
    fig = px.scatter(
        data_frame=data,
        x=[0, 1],
        y=[1, 1],
        color='RGB',
        hover_data=['Color Type', 'LAB', 'Delta-E'],
        labels={'x': '', 'y': ''},
        title='🖌️ Input Color vs Closest ISCC-NBS Color',
    )

    # Update marker size and layout
    fig.update_traces(marker=dict(size=50, line=dict(width=2, color='DarkSlateGrey')))
    fig.update_layout(
        showlegend=False,
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        template='plotly_dark',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    # Add annotations
    fig.add_annotation(
        x=0,
        y=1.05,
        text='Input Color',
        showarrow=False,
        font=dict(size=14, color='white'),
        xanchor='center'
    )
    fig.add_annotation(
        x=1,
        y=1.05,
        text=f'Closest: {closest_color_name}',
        showarrow=False,
        font=dict(size=14, color='white'),
        xanchor='center'
    )

    return fig

def create_interactive_lab_comparison_plot(input_lab, closest_lab, closest_color_name, input_rgb, closest_rgb):
    """
    Creates an interactive Plotly bar plot comparing LAB values with Delta-E indicators.

    Parameters:
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.
        input_rgb (tuple): RGB values of the input color.
        closest_rgb (tuple): RGB values of the closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    components = ['L', 'A', 'B']
    data = pd.DataFrame({
        'Component': components * 2,
        'Value': input_lab + closest_lab,
        'Type': ['Input LAB'] * 3 + [f'Closest LAB: {closest_color_name}'] * 3
    })

    # Define color mapping using actual RGB colors
    color_map = {
        'Input LAB': f'rgb{input_rgb}',
        f'Closest LAB: {closest_color_name}': f'rgb{closest_rgb}'
    }

    fig = px.bar(
        data_frame=data,
        x='Component',
        y='Value',
        color='Type',
        barmode='group',
        hover_data=['Value'],
        title='🔍 LAB Value Comparison',
        template='plotly_dark',
        color_discrete_map=color_map
    )

    # Add Delta-E as annotations above the bars
    for i, component in enumerate(components):
        delta = abs(input_lab[i] - closest_lab[i])
        fig.add_annotation(
            x=component,
            y=max(input_lab[i], closest_lab[i]) + 5,
            text=f'Delta-E: {delta:.2f}',
            showarrow=False,
            font=dict(size=12, color='white')
        )

    fig.update_layout(
        xaxis_title='LAB Components',
        yaxis_title='Values',
        legend_title='Color Type',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    return fig

def create_dynamic_lab_color_space_plot(input_lab, closest_lab, closest_color_name, dataset_df, input_rgb, closest_rgb):
    """
    Creates an enhanced 3D Plotly scatter plot visualizing the input and closest colors along with the dataset.
    Ensures the input color is accurately represented.

    Parameters:
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.
        input_rgb (tuple): RGB values of the input color.
        closest_rgb (tuple): RGB values of the closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    # Prepare dataset points
    dataset_points = go.Scatter3d(
        x=dataset_df['L'],
        y=dataset_df['A'],
        z=dataset_df['B'],
        mode='markers',
        marker=dict(
            size=3,
            color='lightgrey',
            opacity=0.5
        ),
        name='Dataset Colors',
        hoverinfo='text',
        text=dataset_df['Color Name']
    )

    # Define RGB color strings
    input_rgb_str = f'rgb{input_rgb}'
    closest_rgb_str = f'rgb{closest_rgb}'

    # Input Color
    input_point = go.Scatter3d(
        x=[input_lab[0]],
        y=[input_lab[1]],
        z=[input_lab[2]],
        mode='markers+text',
        marker=dict(size=10, color=input_rgb_str, opacity=1),
        text=['Input Color'],
        textposition='top center',
        name='Input Color',
        hoverinfo='text'
    )

    # Closest Color
    closest_point = go.Scatter3d(
        x=[closest_lab[0]],
        y=[closest_lab[1]],
        z=[closest_lab[2]],
        mode='markers+text',
        marker=dict(size=10, color=closest_rgb_str, opacity=1),
        text=[f'Closest: {closest_color_name}'],
        textposition='top center',
        name='Closest Color',
        hoverinfo='text'
    )

    # Create figure
    fig = go.Figure(data=[dataset_points, input_point, closest_point])

    # Layout enhancements
    fig.update_layout(
        title='🌐 Dynamic 3D LAB Color Space Visualization',
        scene=dict(
            xaxis_title='L',
            yaxis_title='A',
            zaxis_title='B',
            xaxis=dict(range=[0, 100], backgroundcolor='rgb(20, 20, 20)'),
            yaxis=dict(range=[-128, 127], backgroundcolor='rgb(20, 20, 20)'),
            zaxis=dict(range=[-128, 127], backgroundcolor='rgb(20, 20, 20)'),
            bgcolor='rgb(20, 20, 20)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        legend=dict(
            x=0.7,
            y=0.9,
            bgcolor='rgba(0,0,0,0)',
            bordercolor='rgba(0,0,0,0)'
        ),
        template='plotly_dark',
        margin=dict(l=0, r=0, t=80, b=0)
    )

    return fig

def create_delta_e_distribution_histogram(delta_e_values):
    """
    Creates a histogram of Delta-E values to visualize the distribution of color differences.

    Parameters:
        delta_e_values (np.ndarray): Array of Delta-E values.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    fig = px.histogram(
        x=delta_e_values,
        nbins=30,
        title='📊 Delta-E Distribution',
        labels={'x': 'Delta-E Value', 'y': 'Count'},
        template='plotly_dark',
        opacity=0.75
    )

    fig.update_layout(
        xaxis=dict(title='Delta-E'),
        yaxis=dict(title='Frequency'),
        margin=dict(l=50, r=50, t=80, b=50)
    )

    return fig

def create_color_density_heatmap(dataset_df):
    """
    Creates a heatmap representing the density of colors in the A-B plane of the LAB space.

    Parameters:
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    fig = px.density_heatmap(
        dataset_df,
        x='A',
        y='B',
        nbinsx=50,
        nbinsy=50,
        title='🔥 Color Density Heatmap in A-B Plane',
        labels={'A': 'A Component', 'B': 'B Component'},
        color_continuous_scale='Viridis',
        template='plotly_dark'
    )

    fig.update_layout(
        xaxis_title='A',
        yaxis_title='B',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    return fig

def create_pairwise_scatter_matrix(dataset_df, input_lab, closest_lab):
    """
    Creates a Scatter Plot Matrix to explore relationships between LAB components.
    Highlights input and closest colors.

    Parameters:
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.

    Returns:
        plotly.graph_objects.Figure: The Splom figure.
    """
    # Create a copy of the dataset
    splom_df = dataset_df.copy()

    # Add input color
    input_row = {
        'L': input_lab[0],
        'A': input_lab[1],
        'B': input_lab[2],
        'Color Name': 'Input Color',
        'Group': 'Input Color'
    }

    # Add closest color
    closest_row = {
        'L': closest_lab[0],
        'A': closest_lab[1],
        'B': closest_lab[2],
        'Color Name': 'Closest Color',
        'Group': 'Closest Color'
    }

    # Append input and closest colors to the dataset using pd.concat
    splom_df = pd.concat([splom_df, pd.DataFrame([input_row, closest_row])], ignore_index=True)

    # Convert input and closest LAB to RGB
    input_rgb = lab_to_rgb(input_lab)
    closest_rgb = lab_to_rgb(closest_lab)

    # Define RGB color strings
    splom_df['Color Group'] = splom_df['Group'].apply(
        lambda x: f'rgb{input_rgb}' if x == 'Input Color' else (f'rgb{closest_rgb}' if x == 'Closest Color' else 'lightgrey')
    )

    # Create Splom trace
    splom_trace = go.Splom(
        dimensions=[
            dict(label='L', values=splom_df['L']),
            dict(label='A', values=splom_df['A']),
            dict(label='B', values=splom_df['B'])
        ],
        text=splom_df['Color Name'],
        marker=dict(
            size=5,
            color=splom_df['Color Group'],
            opacity=0.7
        ),
        diagonal_visible=False,
        showupperhalf=False,
        name='Colors'
    )

    # Create figure with Splom
    fig_splom = go.Figure(data=[splom_trace])

    # Update layout
    fig_splom.update_layout(
        title='🔍 Pairwise LAB Relationships',
        template='plotly_dark',
        dragmode='select',
        height=800
    )

    return fig_splom

def create_combined_visualizations(fig1, fig2, fig3, fig4, fig5):
    """
    Combines all individual plots (excluding 'splom') into a single subplot layout.

    Parameters:
        fig1 to fig5 (plotly.graph_objects.Figure): Individual Plotly figures.

    Returns:
        plotly.graph_objects.Figure: The combined Plotly figure object.
    """
    # Define subplot structure without 'splom'
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Input vs Closest Color',
            'LAB Value Comparison',
            '3D LAB Color Space',
            'Delta-E Distribution',
            'Color Density Heatmap',
            'Pairwise LAB Relationships'  # Handled separately
        ),
        specs=[
            [{"type": "scatter"}, {"type": "bar"}],
            [{"type": "scatter3d"}, {"type": "histogram"}],
            [{"type": "heatmap"}, {"type": "scatter"}]  # Placeholder for 'splom'
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )

    # Add traces to subplots
    for trace in fig1['data']:
        fig.add_trace(trace, row=1, col=1)
    for trace in fig2['data']:
        fig.add_trace(trace, row=1, col=2)
    for trace in fig3['data']:
        fig.add_trace(trace, row=2, col=1)
    for trace in fig4['data']:
        fig.add_trace(trace, row=2, col=2)
    for trace in fig5['data']:
        fig.add_trace(trace, row=3, col=1)
    # 'Pairwise LAB Relationships' handled separately

    # Update layout
    fig.update_layout(
        height=1800,
        showlegend=False,
        template='plotly_dark',
        title_text='🎨 Enhanced LAB Color Analyzer Visualizations'
    )

    return fig

def display_tabbed_visualizations(fig_combined, fig_splom):
    """
    Displays all plots in a tabbed interface using ipywidgets.

    Parameters:
        fig_combined (plotly.graph_objects.Figure): Combined subplots figure.
        fig_splom (plotly.graph_objects.Figure): Splom figure.
    """
    tabs = widgets.Tab()

    # Define tabs and their content
    tab_contents = [
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output()  # Added an extra tab for Splom
    ]

    # Set titles for each tab
    tab_titles = [
        'Input vs Closest Color',
        'LAB Value Comparison',
        '3D LAB Color Space',
        'Delta-E Distribution',
        'Color Density Heatmap',
        'Pairwise LAB Relationships',
        'Scatter Plot Matrix'  # New tab for Splom
    ]

    for i in range(len(tab_contents)):
        tabs.set_title(i, tab_titles[i])

    # Populate each tab with corresponding figure
    with tab_contents[0]:
        fig_combined.data[0].show()  # Input vs Closest Color
    with tab_contents[1]:
        fig_combined.data[1].show()  # LAB Value Comparison
    with tab_contents[2]:
        fig_combined.data[2].show()  # 3D LAB Color Space
    with tab_contents[3]:
        fig_combined.data[3].show()  # Delta-E Distribution
    with tab_contents[4]:
        fig_combined.data[4].show()  # Color Density Heatmap
    with tab_contents[5]:
        # Placeholder or instructions
        print("Please refer to the Scatter Plot Matrix tab for detailed pairwise LAB relationships.")
    with tab_contents[6]:
        fig_splom.show()  # Splom

    tabs.children = tab_contents
    display(tabs)

def display_results_table(results):
    """
    Displays the aggregated results in a table with colored RGB representations.

    Parameters:
        results (dict): Dictionary containing result details.
    """
    df = pd.DataFrame([results])

    # Format Delta-E
    df['Delta-E'] = df['Delta-E'].astype(str)

    # Function to display RGB colors as colored cells
    def color_rgb(cell):
        return f'<div style="background-color:{cell}; width:100px; height:20px;"></div>'

    # Apply coloring to RGB columns
    df['Input RGB'] = df['Input RGB'].apply(lambda x: color_rgb(f'rgb{x[0]}, {x[1]}, {x[2]}'))
    df['Closest RGB'] = df['Closest RGB'].apply(lambda x: color_rgb(f'rgb{x[0]}, {x[1]}, {x[2]}'))

    # Use HTML display to render colored cells
    html = df.to_html(escape=False)
    display(HTML(html))

# ------------------------------
# 4. Interactive User Interface
# ------------------------------

def run_color_analyzer():
    """
    Runs the interactive LAB Color Analyzer with fancy visual enhancements.
    """
    # Upload and validate dataset
    try:
        dataset_df = upload_dataset()
        validate_dataset(dataset_df)
        print("✅ Dataset uploaded and validated successfully.")
        print(f"📊 Dataset Preview:\n{dataset_df.head()}")
    except Exception as e:
        print(f"❌ Error uploading or validating dataset: {e}")
        return

    # Define interactive widgets for LAB input with precise controls
    # Sliders with smaller step sizes
    lab_l_slider = widgets.FloatSlider(
        value=50.0,
        min=0.0,
        max=100.0,
        step=0.01,
        description='L:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )
    lab_a_slider = widgets.FloatSlider(
        value=0.0,
        min=-128.0,
        max=127.0,
        step=0.01,
        description='A:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )
    lab_b_slider = widgets.FloatSlider(
        value=0.0,
        min=-128.0,
        max=127.0,
        step=0.01,
        description='B:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )

    # FloatText widgets for precise input
    lab_l_text = widgets.FloatText(
        value=50.0,
        description='L:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )
    lab_a_text = widgets.FloatText(
        value=0.0,
        description='A:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )
    lab_b_text = widgets.FloatText(
        value=0.0,
        description='B:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )

    # Button to find closest color
    run_button = widgets.Button(
        description='🔍 Find Closest Color',
        button_style='success',
        tooltip='Click to find the closest color',
        icon='search'
    )

    # Output area
    output = widgets.Output()

    # Synchronize sliders and text boxes
    def sync_l_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_l_text.value = round(change['new'], 2)

    def sync_l_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_l_slider.value = round(change['new'], 2)

    def sync_a_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_a_text.value = round(change['new'], 2)

    def sync_a_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_a_slider.value = round(change['new'], 2)

    def sync_b_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_b_text.value = round(change['new'], 2)

    def sync_b_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_b_slider.value = round(change['new'], 2)

    lab_l_slider.observe(sync_l_slider_text, names='value')
    lab_l_text.observe(sync_l_text_slider, names='value')

    lab_a_slider.observe(sync_a_slider_text, names='value')
    lab_a_text.observe(sync_a_text_slider, names='value')

    lab_b_slider.observe(sync_b_slider_text, names='value')
    lab_b_text.observe(sync_b_text_slider, names='value')

    # Organize sliders and text inputs side by side
    lab_l_box = widgets.HBox([lab_l_slider, lab_l_text])
    lab_a_box = widgets.HBox([lab_a_slider, lab_a_text])
    lab_b_box = widgets.HBox([lab_b_slider, lab_b_text])

    # Display widgets
    ui = widgets.VBox([lab_l_box, lab_a_box, lab_b_box, run_button])
    display(ui, output)

    def on_run_button_clicked(b):
        with output:
            clear_output(wait=True)
            input_lab = [lab_l_slider.value, lab_a_slider.value, lab_b_slider.value]
            try:
                validate_lab_color(input_lab)
                print(f"🟢 Input LAB Color: L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}")

                # Find closest color
                closest_color, delta_e = find_closest_color(np.array(input_lab), dataset_df)
                closest_color_name = closest_color['Color Name']
                closest_lab = [closest_color['L'], closest_color['A'], closest_color['B']]

                print(f"🎨 Closest ISCC-NBS Color: {closest_color_name}")
                print(f"📏 Delta-E Value: {delta_e:.2f}")
                print(f"🔍 LAB of Closest Color: L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}")

                # Convert to RGB
                input_rgb = lab_to_rgb(input_lab)
                closest_rgb = lab_to_rgb(closest_lab)

                print(f"🎨 Input RGB Color: {input_rgb}")
                print(f"🎨 Closest RGB Color: {closest_rgb}")

                # Calculate all Delta-E values
                all_delta_e = calculate_delta_e(np.array(input_lab), dataset_df)

                # Create visualizations
                fig1 = create_advanced_color_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_color_name, delta_e)
                fig2 = create_interactive_lab_comparison_plot(input_lab, closest_lab, closest_color_name, input_rgb, closest_rgb)
                fig3 = create_dynamic_lab_color_space_plot(input_lab, closest_lab, closest_color_name, dataset_df, input_rgb, closest_rgb)
                fig4 = create_delta_e_distribution_histogram(all_delta_e)
                fig5 = create_color_density_heatmap(dataset_df)
                fig_splom = create_pairwise_scatter_matrix(dataset_df, input_lab, closest_lab)

                # Combine all visualizations into a single figure (excluding splom)
                combined_fig = create_combined_visualizations(fig1, fig2, fig3, fig4, fig5)
                combined_fig.show()

                # Display Splom separately
                fig_splom.show()

                # Display results in table
                results = {
                    'Input LAB': f"L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}",
                    'Closest ISCC-NBS Color': closest_color_name,
                    'Delta-E': f"{delta_e:.2f}",
                    'Closest LAB': f"L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}",
                    'Input RGB': input_rgb,
                    'Closest RGB': closest_rgb
                }
                display_results_table(results)

            except ValueError as ve:
                print(f"❌ Input validation error: {ve}")
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}")

    run_button.on_click(on_run_button_clicked)

# ------------------------------
# 5. Execute the Analyzer
# ------------------------------

# Call the run_color_analyzer function to start the interactive interface
try:
    run_color_analyzer()
except Exception as e:
    print(f"❌ An error occurred while running the analyzer: {e}")